<a href="https://colab.research.google.com/github/sayan0506/SSD-Custom-Object-Detection-Using-Pytorch/blob/main/SSD_Custom_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Implement SSD for Object Detection using Pytorch**

Here, in this notebook we will try to implement SSD(Single Shot multi-box Detection) algorithm for object detection, this code is inspired from the book - "[Modern Computer Vision with PyTorch](https://www.packtpub.com/product/modern-computer-vision-with-pytorch/9781839213472)". FOr this experiment, we will try to utilize the object detection from image size 300, thus we will use SSD300 variant of the SSD series

## **Install Dependencies**

* [torch_snippets](https://github.com/sizhky/torch_snippets) - contains the utility functions for simple supporting tasks regardinfg tor ch implementations
* [How to run wget quietly](http://oliviertech.com/linux/how-to-run-wget-quietly/#:~:text=The%20wget%20command%20is%20used,are%20writtent%20in%20the%20output.) - wget function is used to download files from given url, and "wget -q", here -q parameter helps to download from wget quietly, so that the output/download status will not print in the console/terminal

In [5]:
# installs torch_snippets and utils quietly
!pip install -q torch_snippets

## **Import Libraries**

In [6]:
import os, collections
from torch_snippets import *
import torch
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt

#### **Environment setup**

Checks whether the torch uses the gpu or not, and the currrent device assign and checks device properties

In [7]:
# asign device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#print(f'Available devices {torch.cuda.get_}')
# torch.cuda.current_device() returns device id of the current device from available list
# torch.cuda.get_device_name(id) returns the device name corresponding to the id
print(f'Current device - {torch.cuda.get_device_properties(torch.cuda.current_device())}')

Current device - _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


## **Data Download**

* Download the bus-truck images dataset for object detection from dropbox link.
* Clone ssd utils - [SSD-utils](https://github.com/sizhky/ssd-utils/)

In [8]:
# project folder
project_path = 'open-images-bus-trucks'

# checks whether the data is downloaded or not 
if not os.path.exists(project_path):
  # download the tar.xz zip file from dropbox
  !wget --quiet https://www.dropbox.com/s/agmzwk95v96ihic/\
open-images-bus-trucks.tar.xz
  # extract the tar file
  !tar -xf open-images-bus-trucks.tar.xz
  # remove the zip file to  save space
  !rm open-images-bus-trucks.tar.xz
  # clone the SSD utils repo
  !git clone https://github.com/sizhky/ssd-utils/

## **Data Preperation**

Data investigation is done here, create dataframes and check the available images and annotation files|

In [46]:
# define data folder paths
data_root = os.path.join('/content/', project_path)
image_root = os.path.join(data_root, 'images')

# load the csv file containing data info in a pandas dataframe
# raw dataframe
df_raw = pd.read_csv(os.path.join(data_root, 'df.csv'))
# copy the dataset
df = df_raw.copy()

**Checking duplicate samples**


In [47]:
# checking the duplicate imageid
df_d = df[df.duplicated('ImageID')]
df_d

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,XClick1X,XClick2X,XClick3X,XClick4X,XClick1Y,XClick2Y,XClick3Y,XClick4Y
2,00006bdb1eb5cd74,xclick,Truck,1,0.702500,0.999167,0.204261,0.409774,1,1,0,0,0,0.849167,0.702500,0.906667,0.999167,0.204261,0.398496,0.409774,0.295739
8,0004d5a9dd44ab6a,xclick,Truck,1,0.094375,0.897500,0.147014,0.934150,0,0,0,0,0,0.365000,0.094375,0.333750,0.897500,0.147014,0.609495,0.934150,0.822358
9,0004d5a9dd44ab6a,xclick,Truck,1,0.860625,0.999375,0.249617,0.390505,1,0,0,0,0,0.992500,0.999375,0.860625,0.921250,0.249617,0.294028,0.325421,0.390505
15,0007eeeabf3c5e5c,xclick,Bus,1,0.620625,0.999375,0.406667,0.647778,1,1,0,0,0,0.920000,0.999375,0.871875,0.620625,0.406667,0.458889,0.647778,0.432222
19,000924a411c24d25,xclick,Bus,1,0.000000,0.086875,0.429644,0.702627,1,1,0,0,0,0.033125,0.086875,0.008750,0.000000,0.429644,0.632270,0.702627,0.636023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24043,ffc0475ac3f403de,xclick,Truck,1,0.626250,0.999375,0.245779,0.999062,1,0,0,0,0,0.826250,0.626250,0.999375,0.999375,0.245779,0.814259,0.999062,0.999062
24045,ffc67982d4790275,xclick,Bus,1,0.456277,0.999134,0.392991,0.735920,0,1,0,0,0,0.568831,0.456277,0.669264,0.999134,0.392991,0.563204,0.735920,0.486859
24050,ffd1093b9f7d3e13,xclick,Bus,1,0.942500,0.999375,0.468105,0.768293,0,1,0,0,0,0.995625,0.942500,0.978750,0.999375,0.468105,0.608818,0.768293,0.583490
24059,fff376d20410e4c9,xclick,Bus,1,0.348125,0.701250,0.423333,0.744167,0,0,0,1,0,0.478750,0.493125,0.348125,0.701250,0.423333,0.744167,0.537500,0.523333


**Define targets**

In [57]:
# this ensures whether the image ids are within unique imageid
df = df[df['ImageID'].isin(df['ImageID'].unique().tolist())]
# create target dictionary from unique labels
label2target = {l:t+1 for t,l in enumerate(df_raw['LabelName'].unique())}
# define additional target 'background' having id  = 0, used in SSD(-ve class)
label2target['background'] = 0
# define target2label dict
target2label = {t:l for l,t in label2target.items()}

num_classes = len(label2target)

print(f'Dataset contains total {num_classes} classes, having below categories {target2label}')
print(label2target)

Dataset contains total 3 classes, having below categories {1: 'Bus', 2: 'Truck', 0: 'background'}
{'Bus': 1, 'Truck': 2, 'background': 0}
